# Process CS3216 Peer Review Responses

This notebook should work with the default `.csv` file downloaded from Google Sheets. Note that the columns will need to be ordered correctly, i.e.

| A1 - Member 1 | Q1 for A1M1 | Q2 for A1M1 | Q3 for A1M1 | Q4 for A1M1 | A1 - Member 2 | ... |
| ------------- | ----------- | ----------- | ----------- | ----------- | ------------- | --- |

Note that in addition to that CSV file, you will need to add another CSV file containing the names and emails of the students, with the format:

| Name | Email |
| ---- | ----- |

In [ ]:
import pandas as pd
import re
import smtplib

from collections import defaultdict
from pprint import pprint
from email.mime.text import MIMEText
from typing import List, Tuple, Union, Any, Optional
from os import mkdir

In [ ]:
# Define constants
TUTOR_NAME = ''
TUTOR_EMAIL = ''
PROF_EMAIL = 'sooyj@comp.nus.edu.sg'
GMAIL_USERNAME = ''
GMAIL_PASSWORD = ''
SHOULD_SEND_EMAIL = False

RESPONSE_CSV_FILE_NAME = 'CS3216 Peer Evaluation 2021 (Responses) - Form responses 1.csv'
RESPONSE_CSV_NAME_COLUMN = 'Your Name'
Q1 = 'THE GOOD - Please write some positive things about working with this individual'
Q2 = 'THE BAD - Please write some not so positive things about working with this individual'
Q3 = 'Please state some areas of improvement that you think would benefit this person'
Q4 = 'Comment on this member (For instructor only)'
RESPONSE_CSV_QUESTIONS = [Q1, Q2, Q3, Q4]
MEMBER_COLUMN_REGEX = r'^A\d - Member \d'

EMAIL_CSV_FILE_NAME = 'Emails.csv'
EMAIL_CSV_NAME_COLUMN = 'Name'
EMAIL_CSV_EMAIL_COLUMN = 'Email'

EMAIL_TEMPLATE = '''Hi {},

The following is feedback from your group members for each assignment.

{}

{}

{}

Cheers,
{}
'''

ASSIGNMENT_TEMPLATE = '''
Assignment {}:

1. Please write some positive things about working with this individual:

{}


2. Please write some not so positive things about working with this individual:

{}


3. Please state some areas of improvement that you think would benefit this person:

{}

'''

FILE_TEMPLATE = '''
Email message for {} ({}):

{}
'''

In [ ]:
email_file = pd.read_csv(EMAIL_CSV_FILE_NAME)

# Student Name -> Email
emails = {name.strip(): email[0].strip() for name, email in email_file.set_index(EMAIL_CSV_NAME_COLUMN).dropna().T.to_dict('list').items()}
pprint(len(emails))
pprint(emails)

In [ ]:
response_file = pd.read_csv(RESPONSE_CSV_FILE_NAME)
display(response_file.head())

In [ ]:
# Receiver Name -> Assignment -> Question -> [Responses]
# This is for the student RECEIVING feedback, not the one giving
feedback = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

# Giver Name -> Receiver Name -> Feedback
# This contains comments for instructors to see
instructor_feedback = defaultdict(dict)

def process_row(row: pd.Series):
  giver: str = row[RESPONSE_CSV_NAME_COLUMN]
  row_seq: List[Tuple[Union[int, str], Any]] = list(row.items())
  i = 0
  while i < len(row_seq):
    data = row_seq[i]
    if not re.match(MEMBER_COLUMN_REGEX, data[0]):
      i += 1
      continue

    receiver = data[1] # Receiving student name
    if pd.isna(receiver):
      # Handles optional Member 5 columns, for teams with 4 members
      i += 5
      continue
    assignment: str = data[0][:2] # 'A1', 'A2', or 'A3'

    # Go through all the responses for this receiver
    for index, question_response in enumerate(row_seq[i+1:i+4]):
      assert question_response[0].startswith(RESPONSE_CSV_QUESTIONS[index]) # Sanity check
      if pd.isna(question_response[1]):
        continue
      feedback[receiver][assignment][f'Q{index + 1}'].append(str(question_response[1]))

    feedback_to_instructor = row_seq[i+4] # Feedback only visible to instructors
    assert feedback_to_instructor[0].startswith(Q4)
    if not pd.isna(feedback_to_instructor[1]):
      instructor_feedback[giver][receiver] = str(feedback_to_instructor[1])
    i += 5

In [ ]:
for _, row in response_file.iterrows():
  process_row(row=row)

pprint(feedback.keys())
pprint(instructor_feedback)

In [ ]:
# Do a quick sanity check
def check_feedback():
  # We might have more emails than students giving feedback, e.g. due to students dropping out
  assert len(feedback) == len(response_file.index) <= len(emails)
  for _, receiver_feedback in feedback.items():
    assert len(receiver_feedback) == 3
    for _, assignment_feedback in receiver_feedback.items():
      assert len(assignment_feedback) >= 3

check_feedback()

In [ ]:
server: Optional[smtplib.SMTP] = None

def init_email_server():
  if not SHOULD_SEND_EMAIL:
    return
  global server
  server = smtplib.SMTP('smtp.gmail.com', 587)
  server.ehlo()
  server.starttls()
  server.ehlo()
  server.login(GMAIL_USERNAME, GMAIL_PASSWORD)

def send_email(email: str, email_content: str):
  if not SHOULD_SEND_EMAIL:
    return
  global server
  assert server is not None
  message = MIMEText(email_content)
  message['Subject'] = 'CS3216 Peer Appraisals'
  message['From'] = f'{TUTOR_NAME} <{TUTOR_EMAIL}>'
  message['To'] = email
  message['Cc'] = PROF_EMAIL
  server.sendmail(TUTOR_EMAIL, [email, PROF_EMAIL], message.as_string())
  print('Sent mail to {}'.format(email))

init_email_server()

In [ ]:
assignments = ['A1', 'A2', 'A3']
questions = ['Q1', 'Q2', 'Q3']

def process_feedback_to_files():
  try:
    mkdir('peer-appraisals')
  except FileExistsError:
    pass
  for receiver, receiver_feedback in feedback.items():
    # [[A1Q1, A1Q2, ...], [A2Q1, ...], ...]
    assignment_feedbacks: List[List[str]] = []
    for assignment in assignments:
      assignment_feedbacks.append(['\n\n'.join(responses) for responses in receiver_feedback[assignment].values()])

    # [A1 content, A2 content, A3 content]
    assignment_contents = [ASSIGNMENT_TEMPLATE.format(index + 1, *assignment_feedback) for index, assignment_feedback in enumerate(assignment_feedbacks)]
    email_content = EMAIL_TEMPLATE.format(receiver, *assignment_contents, TUTOR_NAME)
    file_content = FILE_TEMPLATE.format(receiver, emails[receiver], email_content)

    if SHOULD_SEND_EMAIL:
      print(f'Sending email to {receiver} ({emails[receiver]})')
      send_email(email=emails[receiver], email_content=email_content)

    # Write to file for each person
    f = open('peer-appraisals/{}.txt'.format(receiver.lower().replace(' ', '-')), 'w')
    f.write(file_content)
    f.close()

process_feedback_to_files()

In [ ]:
def process_instructor_feedback_to_file():
  f = open('instructor-feedback.txt', 'w')
  for giver, giver_feedback in instructor_feedback.items():
    f.write(f"{giver}\n")
    for receiver, comment in giver_feedback.items():
      f.write(f"  - {receiver}: {comment}\n")
    f.write("\n")
  f.close()

process_instructor_feedback_to_file()